# Лабораторная работа: Анализ данных, очистка, генерация признаков и масштабирование

Этот ноутбук воспроизводит шаги из проекта:
- Загрузка датасета
- Анализ пропусков (визуализация тепловой карты и диаграммы)
- Поиск аномалий (BoxPlot по числовым признакам)
- Заполнение пропусков и повторная проверка
- Обработка выбросов и повторная проверка
- Генерация новых признаков
- Кодирование категориальных признаков (OHE, бинаризация)
- Стандартизация (Z-score) и нормализация (Min-Max)


In [ ]:
# Импорт и настройка
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import helpers
helpers.setup()


In [ ]:
# Загрузка данных
path = './train.csv'
df = helpers.load_data(path)
df.head()


In [ ]:
# Анализ пропусков: графики и таблицы
helpers.analyze_misses(df)

missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0].head(20)


In [ ]:
# Поиск аномалий (выбросов)
helpers.anomaly_find(df)


In [ ]:
# Заполнение пропусков и повторная проверка
helpers.analyze_misses(df)
df_filled = helpers.fill_missing_value_mode(df)
helpers.analyze_misses(df_filled)


In [ ]:
# Устранение выбросов и повторная проверка
helpers.anomaly_find(df)
df_no_outliers = helpers.remove_outliers(df)
helpers.anomaly_find(df_no_outliers)


In [ ]:
# Генерация новых признаков
feat_df = helpers.generate_features(df)
feat_df.filter(regex='Num_Product_Categories|Has_Prod_Cat_3|Purchase_per_Category|Is_High_Spender').head()


In [ ]:
# Кодирование категориальных признаков (OHE)
ohe_df = helpers.one_hot_encode(df)
ohe_df.head()


In [ ]:
# Бинаризация числовых признаков
bin_df = helpers.binarize_columns(df)
bin_df.filter(regex='_bin$').head()


In [ ]:
# Стандартизация и нормализация
std_df = helpers.standardize(df)
mm_df = helpers.minmax_scale(df)

std_df.filter(regex='_std$').head(), mm_df.filter(regex='_mm$').head()
